# Data Engineering and Data Preprocessing

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy import stats
import numpy as np

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score
import joblib
from sklearn.model_selection import train_test_split, cross_val_score
from lightgbm import LGBMClassifier

In [2]:
# Load the dataset
file_path = "C:/Users/KIIT/Minor Project/synthetic_preterm_3000_final_95.csv"
df = pd.read_csv(file_path)

# Drop low-impact features based on correlation analysis
df_cleaned = df.drop(columns=["STD", "lenght of contraction"])

# Handling Outliers using Z-score
z_scores = np.abs(stats.zscore(df_cleaned))
df_cleaned = df_cleaned[(z_scores < 3).all(axis=1)]  # Keeping data within 3 standard deviations

In [3]:
df_cleaned 

,Count Contraction,Entropy,Contraction times,Pre-term,Risk Factor Score,Minor Health Indicator,Lifestyle Factor
0,11514,0.044455,1.291351,1,0.471035,0.096713,0.071005
1,11539,0.981544,0.424260,1,0.708752,0.101520,0.096091
2,7684,0.324470,1.005407,1,0.925776,0.213598,0.134627
3,10629,0.667819,-0.940382,0,0.109835,0.105040,0.192961
4,9887,0.335620,0.520655,0,-0.066231,0.054059,0.166894
...,...,...,...,...,...,...,...
2995,12009,-0.010580,0.380057,0,0.041634,0.013795,0.035682
2996,9739,-0.089561,0.202480,0,-0.149681,0.084975,0.066435
2997,12039,0.519564,-0.763530,0,-0.172221,0.039890,0.089791
2998,9730,0.453195,0.951061,1,0.241077,0.158409,0.059210


In [4]:
# Handling Outliers using Z-score
z_scores 

array([[0.88945143, 0.5569339 , 2.19499383, ..., 0.33590759, 0.29446283,
        0.80406115],
       [0.90392651, 1.47534444, 0.28901264, ..., 0.84782005, 0.21409531,
        0.38891305],
       [1.32813043, 0.05033841, 1.56645148, ..., 1.31517188, 1.66003475,
        0.24880745],
       ...,
       [1.19342806, 0.47344119, 2.32190827, ..., 1.04931898, 1.2446519 ,
        0.49317082],
       [0.14349009, 0.32950726, 1.44699239, ..., 0.15929842, 0.7371851 ,
        0.99924182],
       [0.96104246, 0.68981542, 1.81216146, ..., 0.28670032, 0.25191774,
        0.82411177]])

In [5]:
df_cleaned.isnull().sum()

Count Contraction         0
Entropy                   0
Contraction times         0
Pre-term                  0
Risk Factor Score         0
Minor Health Indicator    0
Lifestyle Factor          0
dtype: int64

In [6]:
# Identify numerical and categorical features
numerical_features = df_cleaned.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = df_cleaned.select_dtypes(include=['object']).columns.tolist()

In [7]:
numerical_features

['Count Contraction',
 'Entropy',
 'Contraction times',
 'Pre-term',
 'Risk Factor Score',
 'Minor Health Indicator',
 'Lifestyle Factor']

In [8]:
categorical_features

[]

In [9]:
import pandas as pd
#Removing outlier from length of contraction and Risk Factor Score
# Assuming df is your dataframe
def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df_filtered = df_cleaned[(df_cleaned[column] >= lower_bound) & (df_cleaned[column] <= upper_bound)]
    return df_filtered

df_cleaned = remove_outliers_iqr(df_cleaned, 'Risk Factor Score')

In [10]:
df_cleaned

,Count Contraction,Entropy,Contraction times,Pre-term,Risk Factor Score,Minor Health Indicator,Lifestyle Factor
0,11514,0.044455,1.291351,1,0.471035,0.096713,0.071005
1,11539,0.981544,0.424260,1,0.708752,0.101520,0.096091
2,7684,0.324470,1.005407,1,0.925776,0.213598,0.134627
3,10629,0.667819,-0.940382,0,0.109835,0.105040,0.192961
4,9887,0.335620,0.520655,0,-0.066231,0.054059,0.166894
...,...,...,...,...,...,...,...
2995,12009,-0.010580,0.380057,0,0.041634,0.013795,0.035682
2996,9739,-0.089561,0.202480,0,-0.149681,0.084975,0.066435
2997,12039,0.519564,-0.763530,0,-0.172221,0.039890,0.089791
2998,9730,0.453195,0.951061,1,0.241077,0.158409,0.059210


In [11]:
df_cleaned['Contraction_Risk_Interaction'] = df['Count Contraction'] * df['Risk Factor Score']

C:\Users\KIIT\AppData\Local\Temp\ipykernel_23820\203085027.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Contraction_Risk_Interaction'] = df['Count Contraction'] * df['Risk Factor Score']


In [12]:
df_cleaned

,Count Contraction,Entropy,Contraction times,Pre-term,Risk Factor Score,Minor Health Indicator,Lifestyle Factor,Contraction_Risk_Interaction
0,11514,0.044455,1.291351,1,0.471035,0.096713,0.071005,5423.501632
1,11539,0.981544,0.424260,1,0.708752,0.101520,0.096091,8178.286519
2,7684,0.324470,1.005407,1,0.925776,0.213598,0.134627,7113.659200
3,10629,0.667819,-0.940382,0,0.109835,0.105040,0.192961,1167.434579
4,9887,0.335620,0.520655,0,-0.066231,0.054059,0.166894,-654.830643
...,...,...,...,...,...,...,...,...
2995,12009,-0.010580,0.380057,0,0.041634,0.013795,0.035682,499.980007
2996,9739,-0.089561,0.202480,0,-0.149681,0.084975,0.066435,-1457.746853
2997,12039,0.519564,-0.763530,0,-0.172221,0.039890,0.089791,-2073.369642
2998,9730,0.453195,0.951061,1,0.241077,0.158409,0.059210,2345.679290


In [13]:
numerical_cols = df_cleaned.select_dtypes(include=['int64', 'float64']).columns.tolist()

## Pipelines for Scaling

In [14]:
numerical_cols = [col for col in df_cleaned.columns if col not in ["Pre-term"]]

# Define transformation: Only scale numerical features
num_transformer = ColumnTransformer(
    transformers=[("scaler", StandardScaler(), numerical_cols)], 
    remainder="passthrough"  # Keeps 'Pre-term' unchanged
)

# Create pipeline
pipeline = Pipeline(steps=[("scaling", num_transformer)])

# Fit and transform data
df_transformed = pipeline.fit_transform(df_cleaned)

# Convert back to DataFrame with correct column names
df_scaled = pd.DataFrame(df_transformed, columns=numerical_cols + ["Pre-term"])

# Ensure 'Pre-term' is still an integer (in case it was converted)
df_scaled["Pre-term"] = df_cleaned["Pre-term"].values  

# Display transformed data
print(df_scaled.head())

   Count Contraction   Entropy  Contraction times  Risk Factor Score  \
0           0.887630 -0.561523           2.210457           0.336371   
1           0.902108  1.474840           0.287612           0.849471   
2          -1.330419  0.046970           1.576354           1.317907   
3           0.375104  0.793093          -2.738593          -0.443265   
4          -0.054607  0.071201           0.501377          -0.823297   

   Minor Health Indicator  Lifestyle Factor  Contraction_Risk_Interaction  \
0               -0.296316         -0.804370                      0.482332   
1               -0.215963         -0.389328                      1.069699   
2                1.657814          0.248231                      0.842702   
3               -0.157102          1.213328                     -0.425134   
4               -1.009442          0.782064                     -0.813672   

   Pre-term  
0         1  
1         1  
2         1  
3         0  
4         0  


In [15]:
num_transformer

C:\Users\KIIT\AppData\Roaming\Python\Python311\site-packages\sklearn\compose\_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


ColumnTransformer(remainder='passthrough',
                  transformers=[('scaler', StandardScaler(),
                                 ['Count Contraction', 'Entropy',
                                  'Contraction times', 'Risk Factor Score',
                                  'Minor Health Indicator', 'Lifestyle Factor',
                                  'Contraction_Risk_Interaction'])])

In [16]:
df_scaled

,Count Contraction,Entropy,Contraction times,Risk Factor Score,Minor Health Indicator,Lifestyle Factor,Contraction_Risk_Interaction,Pre-term
0,0.887630,-0.561523,2.210457,0.336371,-0.296316,-0.804370,0.482332,1
1,0.902108,1.474840,0.287612,0.849471,-0.215963,-0.389328,1.069699,1
2,-1.330419,0.046970,1.576354,1.317907,1.657814,0.248231,0.842702,1
3,0.375104,0.793093,-2.738593,-0.443265,-0.157102,1.213328,-0.425134,0
4,-0.054607,0.071201,0.501377,-0.823297,-1.009442,0.782064,-0.813672,0
...,...,...,...,...,...,...,...,...
2986,1.174297,-0.681117,0.189590,-0.590474,-1.682595,-1.388759,-0.567446,0
2987,-0.140317,-0.852751,-0.204202,-1.003420,-0.492559,-0.879981,-0.984867,0
2988,1.191671,0.470923,-2.346410,-1.052071,-1.246326,-0.493559,-1.116128,0
2989,-0.145529,0.326700,1.455837,-0.159985,0.735138,-0.999501,-0.173912,1


## Key Takeaways:-

    #   Removed outliers from 'length of contraction' & 'Risk Factor Score' using IQR method.
    #  Created a new prameter Contraction_Risk_Interaction